## Load libraries

In [42]:
# tensorflow backend
%load_ext autoreload
%autoreload 2
from os import environ
environ['KERAS_BACKEND'] = 'tensorflow'
# vae stuff
from chemvae.vae_utils import VAEUtils
from chemvae import mol_utils as mu
# import scientific py
import numpy as np
import pandas as pd
# rdkit stuff
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import PandasTools
# plotting stuff
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import SVG, display
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load a model

In [45]:
# vae = VAEUtils(directory='../models/MP_properties')
# vae = VAEUtils(directory='../models/MP_properties/')
import os

os.chdir('/Users/apple/Desktop/鋰電池電解液/VAE_molecule/chemical_vae/models/MP_properties/')
# print(os.getcwd())
vae = VAEUtils(directory='../models/MP_properties/')

hellow world
Using standarized functions? True
Standarization: estimating mu and std values ...the program had really come to here!!
(15185, 120, 35)
(15185, 196)
done!


# Using the VAE


## Decode/Encode 

Might not be perfect (it's probabilistic), try it several times.

smiles <i class="fa fa-arrow-right" aria-hidden="true"></i> x <i class="fa fa-arrow-right" aria-hidden="true"></i> z <i class="fa fa-arrow-right" aria-hidden="true"></i> x_r <i class="fa fa-arrow-right" aria-hidden="true"></i> smiles_r

In [61]:
smiles_1 = mu.canon_smiles('C1(N(C(CC(=O)C1)(C)C)O)(C)C')

for i in range(25):
    X_1 = vae.smiles_to_hot(smiles_1,canonize_smiles=False)
    z_1 = vae.encode(X_1)
    X_r= vae.decode(z_1)

#     print('{:25s} : {}'.format('Input',smiles_1))
    print('{:20s} : {}'.format('Reconstruction',vae.hot_to_smiles(X_r,strip=True)[0]))

#     print('{:20s} : {} with norm {:.3f}'.format('Z representation',z_1.shape, np.linalg.norm(z_1)))


Reconstruction       : C(Cc(C)Cc))(2c1(CCNccOC)C)O                     c
Reconstruction       : N1O2[CC(c]1=))(ccc2)(c2c1                                                       )
Reconstruction       : CCOC(cc1)C)cCc)cccCNc1cc2
Reconstruction       : C[CC)CS(Cc1c(CccOc1)cc(Cc
Reconstruction       : cCc1(cC)cO((C@H)N(CCcc)O
Reconstruction       : C1=C(@OCC=1[C1OC)c(C1c1cO
Reconstruction       : C(C=Ccc1)C@]CO2CN(c1OCcS)                                                                               c
Reconstruction       : c1=Ccc1c)(=CC)(NcN)Cc)CNOC
Reconstruction       : C1Cc))cccCN[cn1c1c2)1Cc1)                             2       2  c
Reconstruction       : C(=C(C)1cN)c)c2c2)ccc)CS1
Reconstruction       : CC(1[)=Cc1C2O@H)(CCO2Occ                                                                                      ) c
Reconstruction       : CN)(C)C(=CcOcc)=c1=c1c1Nc N
Reconstruction       : c1c(CC\1cc))c1)c(Nc1OC)))   )
Reconstruction       : c1(=C(((C=C[)OC)c2N)ccc)1                    

## property preditor

In [220]:
print('Properties (qed,SAS,logP):')
y_1 = vae.predict_prop_Z(z_1)
print(y_1)

Properties (qed,SAS,logP):
[[3.1159394]]


In [245]:
# this is the optimize process in the latent space.
%load_ext autoreload
%autoreload 2
from scipy import optimize
import numpy as np 
for i in range(
    print(str(i)+"_attemp")
    initial = np.random.rand(1,196)
    print(initial.shape)
    res1 = optimize.fmin_cg(vae.predict_prop_Z, initial,full_output=True)
    y_1 = vae.predict_prop_Z(initial)
    print(y_1)
# 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0_attemp
(1, 196)
x0 shape is(196,)
         Current function value: 2.921356
         Iterations: 1
         Function evaluations: 27986
         Gradient evaluations: 71
[[2.9213564]]


## Decode several attempts
VAE are probabilistic

In [55]:
noise=5.0
print('Searching molecules randomly sampled from {:.2f} std (z-distance) from the point'.format(noise))
df = vae.z_to_smiles( z_1,decode_attempts=100,noise_norm=noise)
print('Found {:d} unique mols, out of {:d}'.format(len(set(df['smiles'])),sum(df['count'])))
print('SMILES\n',df.smiles)
# display(PandasTools.FrameToGridImage(df,column='mol', legendsCol='smiles',molsPerRow=5))
df.head()

Searching molecules randomly sampled from 5.00 std (z-distance) from the point
Found 0 unique mols, out of 0
SMILES
 Series([], Name: smiles, dtype: object)


,smiles,count,mol


## PCA of latent space

Sample random points from the training set along with properties

In [70]:
Z, data, smiles = vae.ls_sampler_w_prop(size=5000,return_smiles=True)
prop_opt = 'qed'
prop_df = pd.DataFrame(data).reset_index()
prop_df['smiles']=smiles
prop_df.head()

,index,EA,IE,smiles
0,7608,0.098372,4.360567,COCCOCCOc1c(ccc2c1ONO2)c1ccccc1
1,4465,1.754691,3.909977,COCCOCCOCCOn1c2cccc(c2[nH]s1)N
2,8688,1.006411,2.222106,C#Cc1c/c(=c/2\ccn(C)c(c2)C(=O)O)/ccn1C
3,10559,0.383937,2.301879,CN(C)Cn1ccc(=c2ccn(C)cc2)c(c1)C(=O)O
4,13355,-1.210131,3.800791,Cn1ccc(c1)OC


Perform a PCA projection and color the points based on a property

In [72]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# do pca and normalize
Z_pca = PCA(n_components=2).fit_transform(Z)
Z_pca = MinMaxScaler().fit_transform(Z_pca)

df = pd.DataFrame(np.transpose((Z_pca[:,0],Z_pca[:,1])))
df.columns = ['x','y']
df[prop_opt]=prop_df[prop_opt]

plt.scatter(x=df['x'], y=df['y'], c=df[prop_opt],
            cmap= 'viridis', marker='.',
            s=10,alpha=0.5, edgecolors='none')
plt.show()

KeyError: 'qed'

compare with t-SNE, will take some time

In [73]:
from sklearn.manifold import TSNE

Z_tsne = TSNE(n_components=2).fit_transform(Z)
Z_tsne = MinMaxScaler().fit_transform(Z_tsne)

f = pd.DataFrame(np.transpose((Z_tsne[:,0],Z_tsne[:,1])))
df.columns = ['x','y']
df[prop_opt]=prop_df[prop_opt]

plt.scatter(x=df['x'], y=df['y'], c=df[prop_opt],
            cmap= 'viridis', marker='.',
            s=10,alpha=0.5, edgecolors='none')
plt.show()

KeyError: 'qed'